In [ ]:
import seq2seq_LSTM
from keras.models import load_model

%pylab inline

## Specify the number of loops And Locate the bassline and drum dataset files

## **Same files as the ones used for training**

In [ ]:
nbars = 2 # <<<<<<<<<<<<<<<<<<<<

bassline_dataset_file = "../data/bassline_size_50[WithOffet]_translated_to_midi_36-"+str(int(nbars))+"bars.txt"
drum_dataset_file = "../data/drum_size_50.txt"

## Create drum and bassline tokens ( DO NOT MODIFY THE CODE)

In [ ]:
drum_dataset_0b, drum_dataset_int, bassline_dataset = seq2seq_LSTM.get_sequences(bassline_dataset_file, drum_dataset_file)

output_cardinality = len(np.unique(bassline_dataset))+2
input_cardinality = 2**8

print("Bassline Features: ", np.unique(bassline_dataset))

# configure problem
n_features_drums = input_cardinality           # 2^8 different unique drum combinations #Cardinality of Input
n_features_bassline = output_cardinality        #Cardinality of Output
n_units = 128

encoder_input, decoder_input, decoder_output, drum_dataset_int_words_tokens, target_words_tokens, target_in_words_tokens = seq2seq_LSTM.get_dataset(drum_dataset_int, 
                                                                                                                                                    bassline_dataset, 
                                                                                                                                                    padding_value = "S", 
                                                                                                                                                    input_cardinality=input_cardinality, 
                                                                                                                                                    output_cardinality=output_cardinality+1)

print ("drum_dataset_int_words_tokens", drum_dataset_int_words_tokens)
print ("target_words_tokens", target_words_tokens)
print ("target_in_words_tokens", target_in_words_tokens)


## Select the model to be used for prediction

In [ ]:
epochs = 4000

In [ ]:
infenc=load_model("infenc_epoch_"+str(epochs)+"_InputCar_"+str(input_cardinality)+
                  "_OutputCar_"+str(output_cardinality)+"_n_units_"+str(n_units)+".h5")
infdec=load_model("infdec_epoch_"+str(epochs)+"_InputCar_"+str(input_cardinality)+
                  "_OutputCar_"+str(output_cardinality)+"_n_units_"+str(n_units)+".h5")

## Load the test files: Drums for which basslines are to be generated

In [ ]:
# get test files
drum_text_txt = "../data_test/drum_size_15.txt"
drum_testset_0b, drum_testset_int = seq2seq_LSTM.get_drum_sequences(drum_dataset_file=drum_text_txt)
print(drum_testset_0b)

## Predict

### sample_mode: 
   #### Mode 0: Select the highest probable output
   #### Mode 1: Select based on the probability distribution (as is, not scaled)
   #### Mode 2: Scale the distribution using temperature and output most probable one

In [ ]:
for k in range(15):
    sample_mode = 2
    temperature = 2
    n_steps = nbars*16

    print(drum_dataset_int_words_tokens)

    source_Vectorized = seq2seq_LSTM.Vectorize(drum_testset_int[k], drum_dataset_int_words_tokens)

    source = seq2seq_LSTM.to_categorical([source_Vectorized], num_classes=input_cardinality)

    prediction = seq2seq_LSTM.predict_sequence(infenc, infdec, source, n_steps=n_steps, cardinality=output_cardinality)

    drum_dataset_int_tokens_words = {v: k for k, v in target_words_tokens.items()}

    output_encoded = seq2seq_LSTM.one_hot_decode(prediction)

    bassline_array = seq2seq_LSTM.one_hot_to_bassline(prediction,target_words_tokens, 
                                                      sample_mode=sample_mode, 
                                                      temperature=temperature)
    print(bassline_array)

    if sample_mode in [0, 1]:
        midi_filename = "../data_test/"+str(k+1)+"/epoch_"+str(epochs)+"_InputCar_"+str(input_cardinality)+"_OutputCar_"+str(output_cardinality)+"_n_units_"+str(n_units)+".mid"
    if sample_mode in [2, 3]:
        midi_filename = "../data_test/"+str(k+1)+"/epoch_"+str(epochs)+"_InputCar_"+str(input_cardinality)+"_OutputCar_"+str(output_cardinality)+"_n_units_"+str(n_units)+"_temperature_"+str(temperature)+".mid"

    seq2seq_LSTM.bassline_to_midi(bassline_array, 
                                  midi_filename, 
                                  note_duration=.25)
